In [1]:
import pyspark as ps
import pandas as pd
import re

In [11]:
pwd

'/home/jovyan/work'

In [66]:
#df = pd.read_csv('work/data/articles1.csv')

In [74]:
#df = df['content']

In [75]:
#df.to_csv('work/data/art1.csv')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [2]:
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('news')
         .getOrCreate()
        )

In [12]:
df = spark.read.load('/home/jovyan/work/data/art1.parquet',
     inferSchema='true', header='true')

In [8]:
#spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [13]:
df.columns

['id', 'content']

In [14]:
from pyspark.sql.functions import *

In [15]:
df = df.select(df['id'], split(df['content'], '[?.,—!;:\’\'"“”\(\)]').alias('content'))

In [40]:
#df = df.select(df['id'], explode(col('content')).alias('content'))

In [43]:
#df = df.select(df['id'], trim(col('content')).alias('content'))

In [44]:
#df = df.select(df['id'], split(col('content'), ' ').alias('content'))

In [4]:
sc = spark.sparkContext

In [5]:
file_rdd = sc.textFile('work/data/art1.csv')

In [8]:
bigrams = file_rdd.flatMap(lambda line: re.split('[?.,—!;:\’\'"“”\(\)]', line)).map(lambda line: line.strip().split(" ")).flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:])))
#bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [9]:
bigrams.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/jovyan/work/work/data/art1.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$

In [9]:
bigrams = bigrams.filter(lambda x: (len(x[0])>0)&(len(x[1])>0))

In [11]:
# import the many data types
from pyspark.sql.types import StructType, StringType, StructField

# create a schema of your own
schema = StructType( [
    StructField('w1',StringType(),True),
    StructField('w2',StringType(),True)] )

In [35]:
df = spark.createDataFrame(bigrams, schema)

In [36]:
df = df.select(col('w1'), col('w2'), concat(col('w1'), lit(' '), col('w2')).alias('phrase'))

In [37]:
df.distinct()

DataFrame[w1: string, w2: string, phrase: string]

[('Congressional', 'Republicans'),
 ('Republicans', 'have'),
 ('have', 'a'),
 ('a', 'new'),
 ('new', 'fear')]